# Sky View Factor

## 0. Initialization
Importing all necessary libraries and specifying the inputs

In [1]:
import os
import topogenesis as tg
import pyvista as pv
import trimesh as tm
import numpy as np
from ladybug.sunpath import Sunpath
import pandas as pd

## 1. Import Meshes (context + envelope)

### 1.1. Load Meshes

In [2]:
envelope_path = os.path.relpath('../data/envelope.obj')
context_path = os.path.relpath('../data/svf_streets.obj')

# load the mesh from file
envelope_mesh = tm.load(envelope_path)
context_mesh = tm.load(context_path)

# Check if the mesh is watertight
print(envelope_mesh.is_watertight)
print(context_mesh.is_watertight)

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


True
False


### 1.2. Visualize Meshes (with pyvista)

In [3]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# adding the meshes
p.add_mesh(tri_to_pv(envelope_mesh), color='#abd8ff')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(286.75114176119666, 204.5425612672697, 271.0256798013578),
 (33.72546195983887, -48.483118534088135, 18.0),
 (0.0, 0.0, 1.0)]

## 2. Import Lattice (envelope)

### 2.1. Load the Envelope Lattice

In [4]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/envelope_lowres_10.8_shadow.csv')
envelope_lattice = tg.lattice_from_csv(lattice_path)

### 2.2. Visualize the Context Mesh + Envelope Lattice

In [5]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
envelope_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(288.14909054942024, 205.94051005549323, 270.6236281604279),
 (33.72546195983887, -48.483118534088135, 16.199999570846558),
 (0.0, 0.0, 1.0)]

## 3. Sky Vectors

### 3.1. Compute Sky Vectors

In [6]:
sphere_mesh = tm.creation.icosphere(subdivisions=3, radius=1.0, color=None)
sphere_vectors = np.copy(sphere_mesh.vertices)

sky_vectors = []
for v in sphere_vectors:
    if v[2] > 0.0:
        sky_vectors.append(v)

sky_vectors = np.array(sky_vectors)
print(sky_vectors.shape)

(305, 3)


In [7]:
full_lattice = envelope_lattice * 0 + 1

In [8]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

# Visualize the mesh using pyvista plotter
#######

# initiating the plotter
p = pv.Plotter(notebook=True)

# fast visualization of the lattice
full_lattice.fast_vis(p)

# adding the meshes
p.add_mesh(tri_to_pv(sphere_mesh), color='#aaaaaa')
p.add_mesh(tri_to_pv(context_mesh), color='#aaaaaa')

# add the sun locations, color orange
p.add_points( sky_vectors * 300, color='#0033ff')

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(996.111517599679, 996.111517599679, 1143.4115175519953),
 (0.0, 0.0, 147.29999995231628),
 (0.0, 0.0, 1.0)]

## 4. Compute Intersection of Sky Rays with Context Mesh

### 4.1. Preparing the List of Ray Directions and Origins

In [9]:
# constructing the ray direction from the sky vectors in a numpy array
sky_dirs = -np.array(sky_vectors)
# exract the centroids of the envelope voxels



vox_cens = full_lattice.centroids
ray_dir = []
ray_src = []
for v_cen in vox_cens:
    for s_dir in sky_dirs:
        ray_dir.append(s_dir)
        ray_src.append(v_cen)
        
# converting the list of directions and sources to numpy array
ray_dir = np.array(ray_dir)
ray_src = np.array(ray_src)

print("number of voxels to shoot rays from :",vox_cens.shape)
print("number of rays per each voxel :",sky_dirs.shape)
print("number of rays to be shooted :",ray_src.shape)

number of voxels to shoot rays from : (540, 3)
number of rays per each voxel : (305, 3)
number of rays to be shooted : (164700, 3)


### 4.2. Computing the Intersection

In [10]:
# computing the intersections of rays with the context mesh
tri_id, ray_id = context_mesh.ray.intersects_id(ray_origins=ray_src, ray_directions=ray_dir, multiple_hits=False)

divide by zero encountered in true_divide
invalid value encountered in subtract


In [11]:
len(ray_id)

28715

## 5. Aggregate Simulation Result in the Sky Access Lattice

### 5.1. Compute the percentage of time that each voxel sees the sun

In [12]:
# initializing the hits list full of zeros
hits = [0]*len(ray_dir)
# setting the rays that had an intersection to 1
for id in ray_id:
    hits[id] = 1

sky_count = len(sky_dirs)
vox_count = len(vox_cens)
# initiating the list of ratio
vox_sky_acc = []
# iterate over the voxels
for v_id in range(vox_count):
    # counter for the intersection
    int_count = 0
    # iterate over the sky rays
    for s_id in range(sky_count):
        # computing the ray id from voxel id and sun id
        r_id = v_id * sky_count + s_id

        # summing the intersections
        int_count += hits[r_id]
    
    # computing the percentage of the rays that DID NOT have 
    # an intersection 
    sky_access = int_count/sky_count

    # add the ratio to list
    vox_sky_acc.append(sky_access)


hits = np.array(hits)
vox_sky_acc = np.array(vox_sky_acc)

print(vox_sky_acc)

[0.         0.6557377  0.36393443 0.23606557 0.         0.63278689
 0.37377049 0.26557377 0.         0.6295082  0.36721311 0.24262295
 0.         0.6295082  0.38032787 0.28196721 0.         0.64590164
 0.3704918  0.24262295 0.         0.66229508 0.38032787 0.23934426
 0.         0.67213115 0.38360656 0.24918033 0.         0.61967213
 0.33114754 0.19672131 0.         0.35737705 0.22622951 0.16065574
 0.         0.56393443 0.33442623 0.22622951 0.         0.35409836
 0.30819672 0.25245902 0.         0.3147541  0.27868852 0.21311475
 0.         0.30491803 0.2852459  0.24262295 0.         0.30819672
 0.26557377 0.21639344 0.         0.35409836 0.2852459  0.2295082
 0.         0.38032787 0.30163934 0.23606557 0.         0.44918033
 0.28852459 0.18688525 0.         0.34754098 0.20983607 0.16393443
 0.         0.50491803 0.31803279 0.21967213 0.         0.22622951
 0.26229508 0.23606557 0.         0.18360656 0.19672131 0.19672131
 0.         0.14754098 0.20655738 0.21311475 0.         0.13770

### 5.2. Store sky access information in a Lattice

In [13]:
# getting the condition of all voxels: are they inside the envelop or not
env_all_vox = full_lattice.flatten()

# all voxels sky access
all_vox_sky_acc = []

# v_id: voxel id in the list of only interior voxels
v_id = 0

# for all the voxels, place the interiority condition of each voxel in "vox_in"
for vox_in in env_all_vox:
    # if the voxel was outside...
    if vox_in == True:
        # read its value of sky access and append it to the list of all voxel sky access
        all_vox_sky_acc.append(vox_sky_acc[v_id])
        # add one to the voxel id so the next time we read the next voxel
        v_id += 1
    # if the voxel was not inside... 
    else:
        # add 0.0 for its sky access
        all_vox_sky_acc.append(0.0)

# convert to array
skyacc_array = np.array(all_vox_sky_acc)

# reshape to lattice shape
skyacc_array = skyacc_array.reshape(envelope_lattice.shape)

# convert to lattice
skyacc_lattice = tg.to_lattice(skyacc_array, envelope_lattice)


print(skyacc_lattice.shape)

(15, 9, 4)


### 5.3. Visualize the sky access lattice

In [14]:
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = skyacc_lattice.shape
# The bottom left corner of the data set
grid.origin = skyacc_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = skyacc_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Sky Access"] = skyacc_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])*1.5
p.add_volume(grid, cmap="coolwarm", clim=[0.0, 1.0],opacity=opacity)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(286.1450885910777, 203.9365080971507, 268.6196266312388),
 (33.72546195983887, -48.483118534088135, 16.200000000000003),
 (0.0, 0.0, 1.0)]

## 6. Save Sun Access Lattice into a CSV

In [55]:
# save the sky access latice to csv

csv_path = os.path.relpath('../data/sky_access_lowres.csv')
skyacc_lattice.to_csv(csv_path)

In [15]:
from scipy.interpolate import RegularGridInterpolator

In [16]:
# loading the lattice from csv
lattice_path = os.path.relpath('../data/envelope_highres_3.6_shadow.csv')
highres_env_lattice = tg.lattice_from_csv(lattice_path)
print(highres_env_lattice.shape)

(43, 24, 11)


In [17]:
def interpolate(low_skyacc_lattice, env_lattice):
    # line spaces
    x_space = np.linspace(low_skyacc_lattice.minbound[0], low_skyacc_lattice.maxbound[0],low_skyacc_lattice.shape[0])
    y_space = np.linspace(low_skyacc_lattice.minbound[1], low_skyacc_lattice.maxbound[1],low_skyacc_lattice.shape[1])
    z_space = np.linspace(low_skyacc_lattice.minbound[2], low_skyacc_lattice.maxbound[2],low_skyacc_lattice.shape[2])

    # interpolation function
    interpolating_function = RegularGridInterpolator((x_space, y_space, z_space), low_skyacc_lattice, bounds_error=False, fill_value=None)

    # high_res lattice
    full_lattice = env_lattice + 1

    # sample points
    sample_points = full_lattice.centroids

    # interpolation
    interpolated_values = interpolating_function(sample_points)

    # lattice construction
    skyacc_lattice = tg.to_lattice(interpolated_values.reshape(env_lattice.shape), env_lattice)

    # nulling the unavailable cells
    skyacc_lattice *= env_lattice
    
    return skyacc_lattice

highres_skyacc_lattice = interpolate(skyacc_lattice,highres_env_lattice)

In [18]:
# convert mesh to pv_mesh
def tri_to_pv(tri_mesh):
    faces = np.pad(tri_mesh.faces, ((0, 0),(1,0)), 'constant', constant_values=3)
    pv_mesh = pv.PolyData(tri_mesh.vertices, faces)
    return pv_mesh

base_lattice = highres_skyacc_lattice
# initiating the plotter
p = pv.Plotter(notebook=True)

# Create the spatial reference
grid = pv.UniformGrid()

# Set the grid dimensions: shape because we want to inject our values
grid.dimensions = base_lattice.shape
# The bottom left corner of the data set
grid.origin = base_lattice.minbound
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit

# Add the data values to the cell data
grid.point_arrays["Sky Access"] = base_lattice.flatten(order="F")  # Flatten the Lattice

# adding the meshes
# p.add_mesh(tri_to_pv(context_mesh), opacity=0.1, style='wireframe')
    
# adding the volume
opacity = np.array([0,0.6,0.6,0.6,0.6,0.6,0.6])
p.add_volume(grid, cmap="coolwarm", clim=[0.0, 1.0],opacity=opacity)

# plotting
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(232.42058333192392, 140.62058333192388, 214.4205833319239),
 (36.000000000000014, -55.800000000000004, 18.0),
 (0.0, 0.0, 1.0)]

In [60]:
# save the sky access latice to csv
csv_path = os.path.relpath('../data/sky_access_highres.csv')
highres_skyacc_lattice.to_csv(csv_path)

## 7. Envelope selection

In [19]:
# extra import function
def lattice_from_csv(file_path):
    # read metadata
    meta_df = pd.read_csv(file_path, nrows=3)

    shape = np.array(meta_df['shape'])
    unit = np.array(meta_df['unit'])
    minbound = np.array(meta_df['minbound'])

    # read lattice
    lattice_df = pd.read_csv(file_path, skiprows=5)

    # create the buffer
    buffer = np.array(lattice_df['value']).reshape(shape)

    # create the lattice
    l = tg.to_lattice(buffer, minbound=minbound, unit=unit)

    return l

In [20]:
# loading the lattice from csv
sky_acc_path = os.path.relpath('../data/sky_access_highres.csv')
sky_acc_lattice = lattice_from_csv(sky_acc_path)

### 7.1. Visualizing the selection

In [21]:
p = pv.Plotter(notebook=True)

base_lattice = sky_acc_lattice

# Set the grid dimensions: shape + 1 because we want to inject our values on the CELL data
grid = pv.UniformGrid()
grid.dimensions = np.array(base_lattice.shape) + 1
# The bottom left corner of the data set
grid.origin = base_lattice.minbound - base_lattice.unit * 0.5
# These are the cell sizes along each axis
grid.spacing = base_lattice.unit 

def create_mesh(value):

    lattice = np.copy(sky_acc_lattice)
    lattice[sky_acc_lattice > value] *= 0.0
    # Add the data values to the cell data
    grid.cell_arrays["Agents"] = lattice.flatten(order="F")  # Flatten the array!
    # filtering the voxels
    threshed = grid.threshold([0.001, 1.0])
    # adding the voxels
    p.add_mesh(threshed, name='sphere', show_edges=True, opacity=1.0, show_scalar_bar=False, clim=[0.0, 1.0])

    return

p.add_slider_widget(create_mesh, [0, 1], title='', value=1.0, event_type="always", style="classic", pointa=(0.1, 0.1), pointb=(0.9, 0.1))
p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(228.1888554793267, 134.58885319050833, 210.18885392960593),
 (36.00000190734863, -57.60000038146973, 18.00000035762787),
 (0.0, 0.0, 1.0)]

### 7.2. Generating an envelope based on the selection

In [23]:
lower_bound = 0.01
upper_bound = 0.3
lower_condition = sky_acc_lattice > lower_bound
upper_condition = sky_acc_lattice < upper_bound
new_avail_lattice = lower_condition * upper_condition

### 7.3. Visualize the new available lattice

In [24]:
p = pv.Plotter(notebook=True)

# adding the avilability lattice
new_avail_lattice.fast_vis(p)

p.show(use_ipyvtk=True)

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

[(238.60252434688636, 146.8025241561515, 220.6025239892585),
 (36.0, -55.80000019073486, 17.99999964237213),
 (0.0, 0.0, 1.0)]

### 7.4. Save new envelope to CSV

In [68]:
csv_path = os.path.relpath('../data/envelope_highres_3.6_svf.csv')
new_avail_lattice.to_csv(csv_path)

### Credits

In [ ]:
__author__ = "Shervin Azadi and Pirouz Nourian"
__license__ = "MIT"
__version__ = "1.0"
__url__ = "https://github.com/shervinazadi/spatial_computing_workshops"
__summary__ = "Spatial Computing Design Studio Workshop on Solar Envelope"